In [1]:
import functools
import pprint as pp
import shlex
import os
import inspect
import ast
from subprocess import run

In [2]:
def memory_profile(import_classes=""):
    def wrapper(f):
        def inner(*args, **kwargs):

            def unwrap_source_code(g):
                sourcelines = inspect.getsourcelines(g)
                for idx, i in enumerate(sourcelines[0]):
                    if i.startswith('def'):            
                        return "".join(sourcelines[0][idx:])
            
            # Writing the f to a temporary file.
            
            temp = './temp_ram.py'
            with open(temp, 'w+') as g:
                decorator = '@profile'
                value = f"""{import_classes}\nfrom memory_profiler import profile\nimport ast\nimport argparse\nimport sys\nimport json\n\n\n{decorator}\n{unwrap_source_code(f)}"""
                g.write(value)
                g.write('\n')

                main_exec = f"""if __name__ == "__main__":     
        parser = argparse.ArgumentParser()
        parser.add_argument('-dummy_pos', nargs='*')
        parser.add_argument('--dummy_optional', nargs="?")    
        args = parser.parse_args()

        star_args = args.dummy_pos
        optional_args = args.dummy_optional

        if optional_args:
            optional_args = optional_args.replace("\'", '"')
            optional_args = json.loads(json.loads(json.dumps(optional_args)))
        else:
            optional_args = {{}} 

        dummy_args = []
        for arg in star_args:
            if arg.isnumeric():
                dummy_args.append(ast.literal_eval(arg))
            else:
                dummy_args.append(arg)


        {f.__name__}(*dummy_args, **optional_args)
                        """
                g.write(main_exec)

            cmd = f'python -m memory_profiler {temp}'
            if args:
                args_split = [f'"{str(i)}"' for i in args]
                second_split =  "-dummy_pos" + " " + " ".join(args_split)

                cmd = cmd + " " + second_split

            if kwargs:
                cmd = cmd + " " + '--dummy_optional' + " " + " ".join([f'"{ {k : str(v) for k, v in kwargs.items()} }"'])

            print(cmd)
            mem_stats = run(cmd, capture_output=True).stdout.decode('utf-8')


            # TODO: We might want to have also 
            # a `debug` argument, where we don't delete
            # the temporary file.

            #os.remove(temp)
            print(mem_stats)
            
            # TODO: f(*args, **kwargs) does not work when we import packages
            # from other modules & needs to be fixed
            
            return f(*args, **kwargs)
        return inner
    return wrapper

    
    
def timing(f):
    import time
    @functools.wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Finished {f.__name__} in {end - start:.4f} seconds.")
        return value
    return inner


    


In [3]:
@memory_profile()
def my_func(x, y):
    a = [1] * (10 ** x)
    b = [2] * (1 * 10 ** y)
    c = a + b
    del c
    return a



my_func(2, 3);

python -m memory_profiler ./temp_ram.py -dummy_pos "2" "3"
Filename: ./temp_ram.py

Line #    Mem usage    Increment   Line Contents
     9     40.3 MiB     40.3 MiB   @profile
    10                             def my_func(x, y):
    11     40.3 MiB      0.0 MiB       a = [1] * (10 ** x)
    12     40.3 MiB      0.0 MiB       b = [2] * (1 * 10 ** y)
    13     40.3 MiB      0.0 MiB       c = a + b
    14     40.3 MiB      0.0 MiB       del c
    15     40.3 MiB      0.0 MiB       return a





In [4]:
@memory_profile()
def custom_string_function(x, y):
    print (x * 2)
    print( y * 3)
    return None

In [5]:
custom_string_function('Thanos', 3)

python -m memory_profiler ./temp_ram.py -dummy_pos "Thanos" "3"
ThanosThanos
9
Filename: ./temp_ram.py

Line #    Mem usage    Increment   Line Contents
     9     40.3 MiB     40.3 MiB   @profile
    10                             def custom_string_function(x, y):
    11     40.3 MiB      0.0 MiB       print (x * 2)
    12     40.3 MiB      0.0 MiB       print( y * 3)
    13     40.3 MiB      0.0 MiB       return None



ThanosThanos
9


In [10]:
from custom_classes import Thanos

@memory_profile("from custom_classes import Thanos")
def test_class_function(a, b):
    obj = Thanos(a)
    res = obj.custom_function(b - 1)
    c = [2] * 10**b
    return res

test_class_function(2, 7);

python -m memory_profiler ./temp_ram.py -dummy_pos "2" "7"
Filename: ./temp_ram.py

Line #    Mem usage    Increment   Line Contents
     9     40.3 MiB     40.3 MiB   @profile
    10                             def test_class_function(a, b):
    11     40.3 MiB      0.0 MiB       obj = Thanos(a)
    12     48.0 MiB      7.6 MiB       res = obj.custom_function(b - 1)
    13    124.3 MiB     76.3 MiB       c = [2] * 10**b
    14    124.3 MiB      0.0 MiB       return res





# External profiler